In [15]:
import random
from tabulate import tabulate

#---------------------------------------------------------------------
#Creating the list for the scenarios grids, valid moves and goal nodes
grids = [[ 
   ['R', 'G', 'G', 'G', 'R', 'G'], 
   ['G', 'G', 'G', 'R', 'G', 'G'], 
   ['G', 'G', 'R', 'G', 'G', 'G'], 
   ['G', 'R', 'G', 'G', 'G', 'R'], 
   ['R', 'G', 'G', 'G', 'R', 'G'], 
   ['G', 'G', 'G', 'R', 'G', 'G'] ],
   [ ['R', 'G', 'R', 'G', 'R', 'G'], 
   ['G', 'R', 'G', 'R', 'G', 'R'], 
   ['R', 'G', 'R', 'G', 'R', 'G'], 
   ['G', 'R', 'G', 'R', 'G', 'R'],
   ['R', 'G', 'R', 'G', 'R', 'G'], 
   ['G', 'R', 'G', 'R', 'G', 'R']
   ]]

moves = [(-1,0), (1,0), (0,-1), (0,1)] 
goal_node = [(2,3), (2,2)]
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#For formatting requirement
B = "\033[1m"
UB = "\033[0m"
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#Function for getting the color positions
def get_color(pos): 
   return grid[pos[0]][pos[1]]
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#Function to caluclate the Heuristic value
def heuristic(node, agent_type): 
   x1, y1 = node 
   x2, y2 = goal 
   color_penalty = 5 if get_color(node) != get_color(goal) else -5 
   return abs(x1 - x2) + abs(y1 - y2) + color_penalty 
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#Function to calculate the Transition Cost
def transition_cost(agent_type, to_node): 
   room_color = get_color(to_node) 
   base = 1 
   if agent_type == 'R1': 
       return base + (10 if room_color == 'G' else -10) 
   elif agent_type == 'G1': 
       return base + (10 if room_color == 'R' else -10)
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#Function to identify the neighbors
def get_neighbors(node):
   neighbors = []
   for dx, dy in moves: 
       nx, ny = node[0] + dx, node[1] + dy 
       if 0 <= nx < rows and 0 <= ny < cols: 
           neighbors.append((nx, ny))
   return neighbors
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#The main algorightm part
def rbfs(node, f_limit, path, g, agent_type, visited): 
   if node == goal: 
       return path + [node], g  # path found with cost 
   successors = [] 
   for neighbor in get_neighbors(node): 
        
       if neighbor in visited: 
           continue 
       g_new = g + transition_cost(agent_type, neighbor) 
       h = heuristic(neighbor, agent_type) 
       f = max(g_new + h, f_limit) 
       successors.append((f, neighbor, g_new)) 
   if not successors: 
       return None, float('inf') 
   successors.sort() 
   while successors: 
       best_f, best_node, best_g = successors[0] 
       if best_f > f_limit: 
           return None, best_f 
       alternative = successors[1][0] if len(successors) > 1 else float('inf') 
       result, best_f_updated = rbfs(best_node, min(f_limit, alternative), path + [node], best_g, agent_type, visited | {best_node}) 
       successors[0] = (best_f_updated, best_node, best_g) 
       successors.sort() 
       if result: 
           return result, best_f_updated 
   return None, float('inf')
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#Function to calculate the total path cost
def calculate_total_path_cost(path, agent_type): 
   if not path or len(path) < 2: 
       return 0 
   cost = 0 
   for i in range(1, len(path)): 
       cost += transition_cost(agent_type, path[i]) 
   return cost
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#Calling the simulation run and appending to the results
def run_all_simulations(): 
   results = []
   for start in start_pos:
       for agent in ['R1', 'G1']: 
           path, _ = rbfs(start, float('inf'), [], 0, agent, {start})
           if path: 
               total_cost = calculate_total_path_cost(path, agent) 
               results.append({ 
                   'Agent': agent, 
                   'Start': start_pos, 
                   'Goal': goal, 
                   'Path': path, 
                   'Steps': len(path) - 1, 
                   'Total Cost': total_cost 
               }) 
           else: 
               results.append({ 
                   'Agent': agent, 
                   'Start': start_pos, 
                   'Goal': goal, 
                   'Path': 'No path found', 
                   'Steps': 'N/A', 
                   'Total Cost': 'N/A' 
               })  
   return results
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#For output summary
def count_colors(grid):
    r_count = sum(row.count('R') for row in grid)
    g_count = sum(row.count('G') for row in grid)
    return {'R': r_count, 'G': g_count}
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#Interpreting the cost incurred by agents in both the scenarios
def analyze_results(grid, results):
    color_counts = count_colors(grid)
    grid_dominance = 'R-dominant' if color_counts['R'] > color_counts['G'] else 'G-dominant' if color_counts['G'] > color_counts['R'] else 'Balanced'

    r1_result = next((r for r in results if r['Agent'] == 'R1'), None)
    g1_result = next((r for r in results if r['Agent'] == 'G1'), None)

    winner = "Tie"
    reason = ""

    if isinstance(r1_result['Total Cost'], int) and isinstance(g1_result['Total Cost'], int):
        if r1_result['Total Cost'] < g1_result['Total Cost']:
            winner = "R1"
            reason = "lower total cost"
        elif r1_result['Total Cost'] > g1_result['Total Cost']:
            winner = "G1"
            reason = "lower total cost"
        else:
            # tie on cost, use steps
            if r1_result['Steps'] < g1_result['Steps']:
                winner = "R1"
                reason = "fewer steps (costs tied)"
            elif r1_result['Steps'] > g1_result['Steps']:
                winner = "G1"
                reason = "fewer steps (costs tied)"
            else:
                winner = "Tie"
                reason = "equal cost and steps"
    elif isinstance(r1_result['Total Cost'], int):
        winner = "R1"
        reason = "G1 failed to find a path"
    elif isinstance(g1_result['Total Cost'], int):
        winner = "G1"
        reason = "R1 failed to find a path"
    else:
        winner = "No agent"
        reason = "No path found"

    return winner, reason, grid_dominance
#---------------------------------------------------------------------

#---------------------------------------------------------------------
#Loop to run the simulation of agents in both the scenarios
for index, grid in enumerate(grids):
    grid = grids[index]
    goal = goal_node [index]
    rows, cols = len(grid), len(grid[0])
    x_cord= random.randint(0, 5)
    y_cord= random.randint(0, 5)
    start_pos = [(x_cord, y_cord)]
    simulation_outputs = run_all_simulations()
    
    print(f"{B}Scenraio -{UB}", index+1, f"{B}:{UB}")
    print("Starting from the node", f"{B}",start_pos,f"{UB}", "towards the Heart in the node at", f"{B}",goal_node [index],f"{UB}"  ":\n")
    headers = ["Agent", "Steps", "Cost", "Path"]
    table_data = []
    for result in simulation_outputs:
        table_data.append([
            result['Agent'],
            result['Steps'],
            result['Total Cost'],
            result['Path']
        ])
    headers = ["Agent", "Steps", "Cost", "Path"]
    bold_headers = [B + h + UB for h in headers]
    print(tabulate(table_data, headers=bold_headers, tablefmt="grid",maxcolwidths=[2, 2,2, 100]))

    winner, reason, dominance = analyze_results(grid, simulation_outputs)
    print(f"{B}Scenraio -{UB}", index+1, f"{B}Analysis :{UB}")
    print(f"  # Grid Type       : {dominance}")
    print(f"  # Best Agent      : {B}{winner}{UB}")
    print(f"  # Reason          : {reason}")
    print("-" * 60)
#---------------------------------------------------------------------

Scenraio - 1 :
Starting from the node  [(4, 0)]  towards the Heart in the node at  (2, 3) :

+---------+---------+--------+------------------------------------------------------------------------------------------------------+
| Agent   |   Steps |   Cost | Path                                                                                                 |
+=========+=========+========+======================================================================================================+
| R1      |      17 |    107 | [(4, 0), (3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (1, 4), (1, |
|         |         |        | 3), (1, 2), (1, 1), (2, 1), (2, 2), (2, 3)]                                                          |
+---------+---------+--------+------------------------------------------------------------------------------------------------------+
| G1      |      17 |    -73 | [(4, 0), (3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4),